In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime as dt
import numpy as np
import re

In [ ]:
propval = pd.read_csv('property_valuation_data.csv')

In [ ]:
#From Data_visualization, we have several columns we can immediately cut

In [ ]:
propval.drop(axis=1, labels = ['DELCHG', 'CORCHG', 'NODESC', 'MBLDG', 'CONDO_S3', 'CONDO_A'], inplace=True)

In [ ]:
propval.drop(axis = 1, labels = 'BBLE', inplace = True)

In [ ]:
propval.columns

In [ ]:
total_entries = len(propval)
valid = []
#For every column of data
for entry in propval.columns:
    counts = propval[entry].value_counts()
    #If it isn't empty:
    if len(counts) > 0:
        #If no one value takes up 80+% of the dataset, it might be worth keeping
        if (counts.iloc[0]/total_entries) <= 0.80:
            #This checks against the data being mostly nans
            if (counts.sum()/total_entries) >= 0.20:
                valid.append(entry)

In [ ]:
len(valid)

In [ ]:
#We've removed 46 columns of mostly-homogenous data.

In [ ]:
propval=propval[valid]

In [ ]:
propval.columns

In [ ]:
#Cut tentative valuations because they're almost always identical to final valuations.

In [ ]:
propval.drop(axis=1, labels = ['TN_AVL', 'TN_AVT', 'TN_EXL', 'TN_AVL_A', 'TN_AVT_A', 'TN_EXL_A', 'TN_EXT_A'], inplace=True)

In [ ]:
propval.columns

In [ ]:
#Superfluous with hnum_hi

In [ ]:
propval.drop(axis=1, labels = ['HNUM_LO'], inplace=True)

In [ ]:
propval.to_csv('property_cleaning_progress.csv')

In [7]:
propval = pd.read_csv('property_cleaning_progress.csv')

C:\Users\marcu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,11,20,22,31,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
basic_columns = propval.columns[1:]

In [9]:
basic_columns

Index(['BORO', 'BLOCK', 'LOT', 'SECVOL', 'DISTRICT', 'YEAR4', 'CUR_FV_L',
       'CUR_FV_T', 'NEW_FV_L', 'NEW_FV_T', 'FV_CHGDT', 'CURAVL', 'CURAVT',
       'CUREXL', 'CUREXT', 'CURAVL_A', 'CURAVT_A', 'CUREXL_A', 'CUREXT_A',
       'CHGDT', 'TN_EXT', 'FCHGDT', 'FN_AVL', 'FN_AVT', 'FN_EXL', 'FN_EXT',
       'FN_AVL_A', 'FN_AVT_A', 'FN_EXL_A', 'FN_EXT_A', 'CBN_TXCL', 'BLDGCL',
       'OWNER', 'HNUM_HI', 'STR_NAME', 'ZIP', 'TOT_UNIT', 'RES_UNIT',
       'LFRT_DEC', 'LDEP_DEC', 'BFRT_DEC', 'BDEP_DEC', 'EXT', 'STORY',
       'LND_AREA', 'GR_SQFT', 'ZONING', 'YRB', 'YRB_FLAG', 'CP_BORO',
       'CP_DIST', 'STATUS1', 'EX_INDS', 'EX_COUNT', 'DCHGDT', 'SM_CHGDT'],
      dtype='object')

In [77]:
#Find everything I need to fix
for entry in basic_columns:
    if propval[entry].isnull().values.any():
            print(entry, 'contains nan')
    if propval[entry].dtype not in ['int64', 'float64']:
        print(entry, propval[entry].dtype)

FV_CHGDT contains nan
FV_CHGDT datetime64[ns]
CHGDT contains nan
CHGDT datetime64[ns]
FCHGDT contains nan
FCHGDT object


KeyError: 'CBN_TXCL'

In [11]:
propval.DISTRICT.unique()

array(['M1', '33', 'B1', '35', '04', '21', '34', '22', '05', '01', '25',
       '57', '26', '27', '52', '87', '42', '24', '23', '41', '03', 'K5',
       'K1', 'K2', '54', '85', 'K4', 'K6', 'K3', 'K7', '06', '43', '53',
       '07', '47', 'K8', '45', '55', 'K9', '02', 'Q1', 'Q2', '44', 'Q3',
       'Q4', 'Q5', 'Q6', '09', 'Q7', '56', 'S1', 'S3', 'S2', 'S4', nan,
       '32', '20', '08', '10', '12', '11', '13', '14', '16', '17', '15',
       '31', '16.0', '31.0', '9.0', '15.0', '17.0', '18.0', '19.0', '2.0',
       '1.0', '5.0', '8.0', '3.0', '4.0', 4.0, 5.0, 6.0, 31.0, 7.0, 9.0,
       10.0, 2.0, 8.0, 1.0, 35.0, 3.0, 36.0, 13.0, 12.0, 14.0, 11.0, 15,
       16.0, 18.0, 17.0, 20.0, 19.0, 22, 21.0, 37.0, '37', 34.0, 32.0,
       33.0, 24.0, 23.0, 25.0, 26.0, 0.0, 27], dtype=object)

In [12]:
dist_replace = {'M1':1, 'B1':1, 'K5':5, 'K1':1, 'K2':2, 'K4':4, 'K6':6, 'K3':3, 'K7':7,\
               'K8':8, 'K9':9, 'Q1':1,'Q2':2, 'Q3':3, 'Q4':4, 'Q5':5, 'Q6':6, 'Q7':7, 'S1':1,\
               'S2':2, 'S3':3, 'S4':4}

In [13]:
new_dist = []
for index, row in propval.DISTRICT.iteritems():
    if row in dist_replace:
        new_dist.append(dist_replace[row])
    elif type(row) == str:
        new_dist.append(float(row))
    else:
        new_dist.append(row)

In [14]:
propval.DISTRICT = new_dist

In [15]:
propval.DISTRICT.unique()

array([ 1., 33., 35.,  4., 21., 34., 22.,  5., 25., 57., 26., 27., 52.,
       87., 42., 24., 23., 41.,  3.,  2., 54., 85.,  6.,  7., 43., 53.,
       47.,  8., 45., 55.,  9., 44., 56., nan, 32., 20., 10., 12., 11.,
       13., 14., 16., 17., 15., 31., 18., 19., 36., 37.,  0.])

In [16]:
propval.DISTRICT.fillna(0, inplace=True)

In [17]:
propval.DISTRICT.dtype

dtype('float64')

In [18]:
propval.FV_CHGDT

0          12/28/2010
1          01/06/2011
2          01/06/2011
3          01/06/2011
4          01/08/2011
              ...    
5738470    12/20/2017
5738471    12/20/2017
5738472    12/20/2017
5738473    12/20/2017
5738474    12/20/2017
Name: FV_CHGDT, Length: 5738475, dtype: object

In [19]:
def convert_datetime(df, column_name):
    return pd.to_datetime(df[column_name], format = '%m/%d/%Y')

In [20]:
#Cleaning up the datetimes into standard format, leaving nans as unknowns

In [21]:
propval.FV_CHGDT = convert_datetime(propval, 'FV_CHGDT')

In [22]:
propval.CHGDT = convert_datetime(propval, 'CHGDT')

In [23]:
propval.CBN_TXCL.fillna(0, inplace = True)
propval.CBN_TXCL.value_counts()

1     4035762
0     1416510
1B     140510
1A     137124
1C       8369
1D        200
Name: CBN_TXCL, dtype: int64

In [24]:
#1C, 1D too small to matter. Others get broken up.
propval['CBN_TXCL1'] = [txcl == '1' for txcl in propval['CBN_TXCL']]
propval['CBN_TXCL0'] = [txcl == '0' for txcl in propval['CBN_TXCL']]
propval['CBN_TXCL1B'] = [txcl == '1B' for txcl in propval['CBN_TXCL']]
propval['CBN_TXCL1A'] = [txcl == '1A' for txcl in propval['CBN_TXCL']]

In [25]:
propval.drop(axis=1, labels = ['CBN_TXCL'], inplace=True)

In [26]:
#Reduce dimensionality
new_bldgcl = []
for index, row in propval.BLDGCL.iteritems():
    new_bldgcl.append(row[:1])

In [27]:
propval['BLDGCL'] = new_bldgcl
propval.BLDGCL.value_counts()

A    2548132
B    2019363
C     596583
R     191442
V     189122
S     167992
G      22423
Z       3418
Name: BLDGCL, dtype: int64

In [28]:
#Cut G, Z take the others
propval['BLDGCLA'] = [bldcl == 'A' for bldcl in propval['BLDGCL']]
propval['BLDGCLB'] = [bldcl == 'B' for bldcl in propval['BLDGCL']]
propval['BLDGCLC'] = [bldcl == 'C' for bldcl in propval['BLDGCL']]
propval['BLDGCLR'] = [bldcl == 'R' for bldcl in propval['BLDGCL']]
propval['BLDGCLV'] = [bldcl == 'V' for bldcl in propval['BLDGCL']]
propval['BLDGCLS'] = [bldcl == 'S' for bldcl in propval['BLDGCL']]

In [29]:
propval.drop(axis=1, labels = ['BLDGCL'], inplace=True)

In [30]:
#Too small, need to cut
propval.drop(axis=1, labels = ['OWNER'], inplace=True)

In [31]:
#Too dimensional, we have other geographical data
propval.drop(axis=1, labels = ['HNUM_HI'], inplace=True)

In [32]:
propval.drop(axis=1, labels = ['STR_NAME'], inplace=True)

In [33]:
propval.EXT.value_counts()

G     2089173
EG     286358
E      254223
Name: EXT, dtype: int64

In [34]:
propval['EXTG'] = [ext == 'G' for ext in propval['EXT']]
propval['EXTEG'] = [ext == 'EG' for ext in propval['EXT']]
propval['EXTE'] = [ext == 'E' for ext in propval['EXT']]

In [35]:
propval.drop(axis=1, labels = ['EXT'], inplace=True)

In [36]:
propval.ZONING.value_counts()[:25]

R5      760347
R4      659494
R3-2    648590
R6      439770
R3-1    396410
R2      365036
R3X     353043
R3A     352032
R4-1    324594
R6B     237167
R2A     236846
R4A     169137
R5B     167169
R1-2    113230
R4B     107420
R7-1     78795
R6A      45141
R5A      27630
M1-1     25183
R7-2     21514
R7A      20188
R5D      14906
R8B      12974
R1-1     12766
R7B      10883
Name: ZONING, dtype: int64

In [37]:
#reduce dimensionality here
new_zoning = []
for index, row in propval.ZONING.iteritems():
    new_zoning.append(str(row)[1])

In [38]:
propval.ZONING=new_zoning

In [39]:
propval.ZONING.value_counts()

3    1762506
4    1279558
5     970355
6     724402
2     613178
1     185529
7     132826
8      32038
a      29873
Z       4478
A       3511
9        173
R         44
B          4
Name: ZONING, dtype: int64

In [40]:
propval.ZONING.replace(to_replace = ['a', 'Z', 'A', 'R', 'B'], value = 0, inplace = True)

In [41]:
propval.ZONING.fillna(0, inplace=True)

In [42]:
new_zoning = []
for index, row in propval.ZONING.iteritems():
    new_zoning.append(int(row))

In [43]:
propval.ZONING = new_zoning

In [44]:
propval.ZONING.value_counts()

3    1762506
4    1279558
5     970355
6     724402
2     613178
1     185529
7     132826
0      37910
8      32038
9        173
Name: ZONING, dtype: int64

In [45]:
propval.YRB_FLAG.value_counts()

E    4445902
Name: YRB_FLAG, dtype: int64

In [46]:
propval.YRB_FLAG.fillna(False, inplace = True)
propval.YRB_FLAG.replace('E', True, inplace=True)

In [48]:
propval.CP_BORO.fillna(0, inplace=True)

In [50]:
propval.CP_DIST.fillna(0, inplace=True)

In [51]:
propval.EX_INDS.value_counts()

EEEE    3302642
EEE      240502
EE       199318
E        194445
E EE      13459
EE E      10241
E  E       4711
E E         524
Name: EX_INDS, dtype: int64

In [52]:
#reduce dimensionality to number of exceptions
propval.EX_INDS.fillna(0, inplace=True)
propval.EX_INDS.replace('EEEE', 4, inplace=True)
propval.EX_INDS.replace(['EEE', 'EE E', 'E EE'], 3, inplace=True)
propval.EX_INDS.replace(['EE', 'E E', 'E  E'], 2, inplace=True)
propval.EX_INDS.replace('E', 1, inplace=True)

In [56]:
propval.DCHGDT = convert_datetime(propval, 'DCHGDT')

In [57]:
propval.SM_CHGDT = convert_datetime(propval, 'SM_CHGDT')

In [58]:
#Fix year zeroes in two ways: replace with mean of borough and with 2 SDs below mean of borough
no_zero = propval[propval.YRB != 0]

In [62]:
YRB1 = np.mean(no_zero[no_zero.BORO == 1].YRB)
YRB2 = np.mean(no_zero[no_zero.BORO == 2].YRB)
YRB3 = np.mean(no_zero[no_zero.BORO == 3].YRB)
YRB4 = np.mean(no_zero[no_zero.BORO == 4].YRB)
YRB5 = np.mean(no_zero[no_zero.BORO == 5].YRB)
STD1 = np.std(no_zero[no_zero.BORO == 1].YRB)
STD2 = np.std(no_zero[no_zero.BORO == 2].YRB)
STD3 = np.std(no_zero[no_zero.BORO == 3].YRB)
STD4 = np.std(no_zero[no_zero.BORO == 4].YRB)
STD5 = np.std(no_zero[no_zero.BORO == 5].YRB)

In [70]:
#Zero-padding so the indicies line up
yrb_means = [0, YRB1, YRB2, YRB3, YRB4, YRB5]
yrb_stds = [0, STD1, STD2, STD3, STD4, STD5]

In [71]:
yrb_means

[0,
 1911.9448359170526,
 1945.396508230097,
 1931.563102943997,
 1940.7330641355545,
 1966.6410247801412]

In [72]:
yrb_new = []
yrb_old = []
for index, row in propval.iterrows():
    if row.YRB == 0:
        yrb_new.append(yrb_means[row.BORO])
        yrb_old.append(yrb_means[row.BORO] - (yrb_stds[row.BORO]*2))
    else:
        yrb_new.append(row.YRB)
        yrb_old.append(row.YRB)

In [73]:
propval['YRB_NEW'] = yrb_new
propval['YRB_OLD'] = yrb_old

In [75]:
propval.YRB_OLD.value_counts()

1920.0    644135
1925.0    533628
1930.0    512917
1950.0    367654
1940.0    297660
           ...  
1798.0         8
1729.0         8
1876.0         8
1816.0         7
1837.0         3
Name: YRB_OLD, Length: 201, dtype: int64

In [76]:
propval.drop(axis=1, labels = ['YRB'], inplace=True)

In [78]:
propval.to_csv('clean_propval_final.csv')

In [79]:
propval = pd.read_csv('clean_propval_final.csv')

C:\Users\marcu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,22,24,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
propval = propval.sort_values(by=['FN_AVT_A'], ascending = False)

In [89]:
propval[['FN_AVT_A', 'BORO']][:25]

,FN_AVT_A,BORO
569048,27468900,4
5598140,25815300,4
4889612,25558848,4
4111139,25558848,4
3402721,25367496,4
2694155,23931600,4
1985872,22576982,4
1277286,21299040,4
1293500,17963591,4
585164,17789265,4
